In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()

Authorization token already provided


In [11]:
from sd3_helper import get_pipeline_options

pt_pipeline_options, use_flash_lora, load_t5 = get_pipeline_options()

display(pt_pipeline_options)

In [4]:
import json
file = open('info.json')
token = json.load(file)['token']

In [12]:
import requests
from pathlib import Path

if not Path("sd3_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/stable-diffusion-v3/sd3_helper.py")
    open("sd3_helper.py", "w").write(r.text)

if not Path("sd3_quantization_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/stable-diffusion-v3/sd3_quantization_helper.py")
    open("sd3_quantization_helper.py", "w").write(r.text)

if not Path("gradio_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/stable-diffusion-v3/gradio_helper.py")
    open("gradio_helper.py", "w").write(r.text)

if not Path("notebook_utils.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py")
    open("notebook_utils.py", "w").write(r.text)

In [13]:
from sd3_helper import OVStableDiffusion3Pipeline, init_pipeline
from notebook_utils import device_widget

device = device_widget()
device

Dropdown(description='Device:', index=3, options=('CPU', 'GPU', 'NPU', 'AUTO'), value='AUTO')

In [15]:
from sd3_helper import TEXT_ENCODER_PATH, TEXT_ENCODER_2_PATH, TEXT_ENCODER_3_PATH, TRANSFORMER_PATH, VAE_DECODER_PATH

models_dict = {"transformer": TRANSFORMER_PATH, "vae": VAE_DECODER_PATH, "text_encoder": TEXT_ENCODER_PATH, "text_encoder_2": TEXT_ENCODER_2_PATH}

if load_t5.value:
    models_dict["text_encoder_3"] = TEXT_ENCODER_3_PATH

ov_pipe = init_pipeline(models_dict, device.value, use_flash_lora.value)

Models compilation


RuntimeError: Exception from src/inference/src/cpp/core.cpp:124:
Exception from src/inference/src/dev/plugin.cpp:58:
Check 'util::directory_exists(path) || util::file_exists(path)' failed at src/frontends/common/src/frontend.cpp:117:
FrontEnd API failed with GeneralFailure:
ir: Could not open the file: "stable-diffusion-3\transformer.xml"




In [ ]:
import torch

image = ov_pipe(
    "A messy room that has been cleaned and items have been rearranged",
    negative_prompt="",
    num_inference_steps=28 if not use_flash_lora.value else 4,
    guidance_scale=5 if not use_flash_lora.value else 0,
    height=512,
    width=512,
    generator=torch.Generator().manual_seed(141),
).images[0]
image

In [ ]:
from notebook_utils import quantization_widget
from sd3_quantization_helper import TRANSFORMER_INT8_PATH, TEXT_ENCODER_INT4_PATH, TEXT_ENCODER_2_INT4_PATH, TEXT_ENCODER_3_INT4_PATH, VAE_DECODER_INT4_PATH

to_quantize = quantization_widget()

to_quantize